## ОБЩИЙ ДЛЯ ВСЕХ РЕШЕНИЙ КОД
---

### НАСТРОЙКА JUPYTER LAB И ПОДКЛЮЧЕНИЕ БИБЛИОТЕК  
---

In [1]:
# Импортировать библиотеки: numpy, pandas pandasql для работы с датасетами
import numpy as np 
import pandas as pd
import pandasql as ps

# Импортировать библиотеку datetime для работы с датой, временем
import datetime as dt

# Импортировать библиотеки pathlib и csv для работы с файлами и импорта csv
from pathlib import Path
import csv

# Импортировать библиотеки hashlib для работы с криптофункциями в том числе с 'md5'
import hashlib as hs

# Настройка среды Jupyter Lab
pd.options.display.max_rows = 400

### БЛОК ИМПОРТА ИСХОДНЫХ ВЕСОВЫХ ДАННЫХ  
---
 - загружаем исходные данные из файла с записями с весовыми показателями и номерами вагонов;
 - чистим загруженные данные;
 - генерируем и добавляем производные данные о массе;
 - на выходе получаем таблицу 'scales'

In [2]:
# Подготовить путь к исходным CSV файлам
dir_path = Path.cwd()

In [4]:
# Загрузить данные из файла *scales*, файл содержит сырые данные по измеренным показателям массы
# номер состава, номер вагона, дата/время, и ряд друхих


# Подготовить переменную с именем импортируемого CSV файла с массами цистерн
file_name_scales = '01__snp-scales.csv'
path_file_scales = Path(dir_path,'res', 'data', file_name_scales)

# Загрузить таблицу с данными взвешивания
with open(path_file_scales, "r", encoding='utf-8') as csv_file_scales:
    scales = pd.read_csv(csv_file_scales,
                         delimiter=';',
                         header=0,
                         names=  ['trnum', 'num', 'bdatetime', 'invnum', 'tcalibr', 'tare', 'brutto', 'netto', 'velocity'] )

# Добавить новый индексный столбец 'id_scales'
scales['id_scales'] = np.arange(2, len(scales['invnum'])+2, 1)

# Зафиксировать количество записей полученных из CSV файла
len_scales = len(scales)

# Привести даты к типу datetime
scales['bdatetime'] = pd.to_datetime(scales['bdatetime'], dayfirst=True, format='%d.%m.%Y %H:%M:%S')

# ЧИСТИМ ДАННЫЕ
# Удалить не нужные столбцы: 'tcalibr', 'netto1', 'velocity'
scales.drop(['tcalibr', 'velocity'], axis=1, inplace=True)

# Удалить строки у которых значение столбца invnum = NaN
scales.dropna(subset=['invnum'], inplace=True)

# Заменить тип столбца 'invnum' на int64
scales['invnum'] = scales['invnum'].astype('int64')

# ГЕНЕРИРОВАТЬ И ДОБАВИТЬ ПРОИЗВОДНЫЕ ДАННЫЕ
# Добавить новый столбец 'deltaweight' содержащий разницу массы на въезде и на выезде
scales['deltaweight'] = scales['brutto'] - scales['tare']

# Задать новый порядок столбцов и проиндексировать
scales=scales.reindex(columns=['id_scales', 'trnum', 'invnum', 'num', 'bdatetime', 'tare', 'brutto', 'netto', 'deltaweight'])


### РАЗБИТЬ ПОЛУЧЕННЫЕ ДАННЫЕ ДВЕ ОСНОВНЫЕ РАБОЧИЕ ТАБЛИЦЫ  
---
 - выделить набор scales_in - записи с составами ВЪЕХАВШИМИ на базу;  
 - выделить набор scales_out - записи с составами ВЫЕХАВШИМИ с базы;
 - разделение провести по значению '0' в поле 'tare'
 - на выходе получаем таблицы 'scales_in' и 'scales_out'

In [5]:
# ВЫделить в отдельную таблицу 'scales_in' записи с транзакцией заехавших составов, по условию 'tare' = 0
scales_in = scales[ scales['tare'] == 0 ]
# ВЫделить в отдельную таблицу 'scales_out' записи с транзакцией выехавших составов, по условию 'tare' != 0
scales_out = scales[ scales['tare'] != 0 ]

# Задать индексацию по 'id_scales'
#scales_in = scales_in.set_index(keys = ['id_scales'], drop=False)
#scales_out = scales_out.set_index(keys = ['id_scales'], drop=False)

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК ТАБЛИЦ 'scales_in' и 'scales_out'  
---
В данном блоке выводятся на печать основные интегральные характеристики полученных таблиц:  
 - количестве записей в каждой из полученных таблиц;
 - типы данных в каждой из итоговых таблиц;
 - количество записей в исходном 'грязном' файле;
 - сумму записей в обеих итоговых таблицах;
 - разница (количество) записей в таблице 'scales_in' и 'scales_out';
 - разница между количеством исходной 'грязной' таблицы и суммой записей двух таблиц, т.е. принципиально bad данные;
 - выведены для визуального контроля первые записи итоговых таблиц;

In [11]:
# Сводная информация по таблице 'scales_in'
scales_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6527 entries, 0 to 12779
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_scales    6527 non-null   int32         
 1   trnum        6527 non-null   int64         
 2   invnum       6527 non-null   int64         
 3   num          6527 non-null   int64         
 4   bdatetime    6527 non-null   datetime64[ns]
 5   tare         6527 non-null   int64         
 6   brutto       6527 non-null   int64         
 7   netto        6527 non-null   int64         
 8   deltaweight  6527 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(7)
memory usage: 484.4 KB


In [11]:
# Сводная информация по таблице 'scales_out'
scales_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6213 entries, 41 to 12739
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_scales    6213 non-null   int32         
 1   trnum        6213 non-null   int64         
 2   invnum       6213 non-null   int64         
 3   num          6213 non-null   int64         
 4   bdatetime    6213 non-null   datetime64[ns]
 5   tare         6213 non-null   int64         
 6   brutto       6213 non-null   int64         
 7   netto        6213 non-null   int64         
 8   deltaweight  6213 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(7)
memory usage: 461.1 KB


In [12]:
# Исходное количество записей полученное из CSV файла до 'чистки' и разделения
len_scales

12780

In [13]:
# Общее количество записей таблицы 'scales_in'
len(scales_in)

6527

In [14]:
# Общее количество записей таблицы 'scales_out'
len(scales_out)

6213

In [15]:
# Сумма записей таблииц 'scales_in' и 'scales_out'
len(scales_in) + len(scales_out)

12740

In [16]:
# Разница записей между таблиицей 'scales_in' и 'scales_out'
len(scales_in) - len(scales_out)

314

In [17]:
# Количество bad значений поля с идентификаторами вагонов 'invnum'
len_scales - (len(scales_in) + len(scales_out))

40

In [18]:
# Процент % bad значений если исходить из того что они равномерно распределены между in и out значениями
(len_scales - (len(scales_in) + len(scales_out))) / (len_scales/100)

0.3129890453834116

In [19]:
# Процент % не состыкованных значени если за 100%, брать количество in записей
(len(scales_in) - len(scales_out)) / (len(scales_in)/100)

4.810785965987437

In [20]:
# Первые значения таблицы 'scales_in'
scales_in.head(2)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight
0,2,761304,73915704,41,2023-01-01 01:41:20,0,27100,0,27100
1,3,761304,50919992,40,2023-01-01 01:41:36,0,26350,0,26350


In [21]:
# Первые значения таблицы 'scales_out'
scales_out.head(2)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight
41,43,761345,57187395,1,2023-01-01 03:35:54,27850,28050,200,200
42,44,761345,54240205,2,2023-01-01 03:36:11,27100,26950,0,-150


### БЛОК ИМПОРТА ИСХОДНЫХ ДАННЫХ ПО ОТБРАКОВКЕ  
---
 - загружаем исходные данные из файла с записями о забракованных вагонов;
 - генерируем и добавляем поле 'id_marriag', которое позволит сопоставлять записи результатов работы программы с исходными 'грязными' данными до обработки
 - чистим загруженные данные;
 - на выходе получаем таблицу 'marriag'

In [44]:
# Загрузить данные из файла *marriag*, файл содержит сырые данные по отбракованным вагонам
# В итоге сосздается dataset 'marriag'

# Подготовить переменную с именем импортируемого CSV файла с дефектными цистернами
file_name_marriag = '02__snp-marriag.csv'
path_file_marriag = Path(dir_path,'res', 'data', file_name_marriag)

# Загрузить таблицу с данными отбраковки
with open(path_file_marriag, "r", encoding='utf-8') as csv_file_marriag:
    marriag = pd.read_csv(csv_file_marriag,
                          delimiter=';',
                          header=0,
                          names=  ['n', 'invnum', 'tcalibr', 'mass', 'nact', 'bdatetime', 'reason', 'contractor'])

# Добавить индексный столбец ID с индексом
marriag['id_marriag'] = np.arange(2, len(marriag['invnum'])+2, 1)

# Удалить не нужные столбцы
marriag.drop(['tcalibr', 'mass'], axis=1, inplace=True)

# Привести даты к типу datetime
marriag['bdatetime'] = pd.to_datetime(marriag['bdatetime'], dayfirst=True, format='%d.%m.%Y')

# Задать новый порядок столбцов и проиндексировать
marriag = marriag.reindex(columns=['invnum', 'id_marriag',  'bdatetime', 'n', 'nact', 'reason', 'contractor'])

# Задать индексацию по 'id_marriag'
marriag = marriag.set_index(keys = ['id_marriag'])

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК ТАБЛИЦЫ 'marriag'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблиц 'marriag':  
 - количестве записей в таблице;
 - типы данных в таблице;
 - выведены для визуального контроля первые записи итоговых таблиц;

In [45]:
# Сводная информация по таблице 'marriag'
marriag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 2 to 442
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   invnum      441 non-null    int64         
 1   bdatetime   441 non-null    datetime64[ns]
 2   n           441 non-null    int64         
 3   nact        441 non-null    int64         
 4   reason      441 non-null    object        
 5   contractor  433 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 24.1+ KB


In [46]:
# Общее количество записей таблицы 'marriag'
len(marriag)

441

In [47]:
# Первые значения таблицы 'marriag'
marriag.head(5)

,invnum,bdatetime,n,nact,reason,contractor
id_marriag,,,,,,
2,75034488,2023-01-01,1,1,прикрытие,"АО""НТС"""
3,51519536,2023-01-01,2,1,прикрытие,ЭКЗА
4,50624089,2023-01-01,3,2,прикрытие,рн-транс
5,53920021,2023-01-01,4,2,прикрытие,рн-транс
6,54770961,2023-01-01,5,2,запрет погрузки,рн-транс


## ВЕТКА 1  
---
РЕШЕНИЕ ЗАДАЧИ ЧЕРЕЗ ЯВНО СУЩЕСТВУЮЩИЕ ПРИЗНАКИ. НЕ ВВОДИТСЯ И НЕ ИСПОЛЬЗУЕТСЯ ПОНЯТИЕ HASH ЗНАЧЕНИЯ

### БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_in' И 'scales_out'
---
В данном блоке производится объединение записей таблиц 'scales_in' и 'scales_out' для получения 'правильных' записей о транзакции состава и всех входящих в него вагонов которые въехали на базу для налива и выехали после. Соединение производится по правилу LEFT JOIN с выполнением тройного условия. Так как 'pandas' не умеет выполнять оьбъединенеия по нечеткому соответствию (а именно таким получается третье условие), то для соединения была задействована дополнительная библиотека 'pandassql' которая работает поверх 'pandas' и перегоняет (динамичесуки) данные в SQL базу данных SQLite, и позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames.  
Условия соединенеия подразумевают совпадения следующих полей обеих DataFrames:  
 - 'invnum';
 - 'num';
 - нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
 - на выходе получаем таблицу 'scales_join';

In [12]:
# Подготовить SQL запрос для левого соединения таблиц 'scales_in' и 'scales_out' с помощью функционала библиотеки 'pandasql'

join_query_scales = '''
    SELECT
        l.id_scales      as id_scales
        ,r.trnum         as trnum_in
        ,l.trnum         as trnum_out
        ,r.invnum        as invnum_in
        ,l.invnum        as invnum_out
        ,r.num           as num_in
        ,l.num           as num_out
        ,r.bdatetime     as date_in
        ,l.bdatetime     as date_out
        ,r.brutto        as tare_in
        ,l.tare          as tare_out
        ,l.brutto        as brutto
        ,l.netto         as netto
        ,l.deltaweight   as deltaweight
    FROM scales_in as r LEFT JOIN scales_out as l
         ON (r.invnum = l.invnum)
            AND (r.num = l.num)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.bdatetime, '-37 hours') as interger) AND cast(strftime('%s',l.bdatetime) as interger) )
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
scales_join = ps.sqldf(join_query_scales, locals())

# Привести даты к типу datetime
scales_join['date_in'] = pd.to_datetime(scales_join['date_in'])
scales_join['date_out'] = pd.to_datetime(scales_join['date_out'])

# Вставить столбец 'deltatetime' с данными разницы времени (дени - часа - минуты - секунды) между событием 'въехал' и событием 'выехал'
scales_join.insert(9, 'deltatetime', scales_join['date_out']-scales_join['date_in'])

# Нормировать разницу к формату 00,00 (часы и доли часа)
scales_join['deltatetime'] = (scales_join['deltatetime']/pd.Timedelta('1 hour')).round(2)

# Преобразовать 'deltatetime' к string и заменить '.' на ',' т.к. импортирует с '.' асурд
scales_join['deltatetime'] = scales_join['deltatetime'].astype(str).str.replace('.', ',', regex=False)
scales_join['deltatetime'] = scales_join['deltatetime'].astype(str).str.replace('nan', '', regex=False)

# Заменить NaN в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на 0
scales_join[[ 'id_scales'
             ,'trnum_out'
             ,'invnum_out'
             ,'num_out'
             ,'deltatetime'
             ,'tare_out'
             ,'brutto'
             ,'netto'
             ,'deltaweight']] = scales_join[['id_scales', 'trnum_out', 'invnum_out', 'num_out' ,'deltatetime', 'tare_out', 'brutto', 'netto', 'deltaweight']].fillna(0)

# Заменить тип в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на int64
scales_join[[ 'id_scales'
             ,'trnum_out'
             ,'invnum_out'
             ,'num_out'
             ,'tare_out'
             ,'brutto'
             ,'netto'
             ,'deltaweight']] = scales_join[['id_scales', 'trnum_out', 'invnum_out', 'num_out', 'tare_out','brutto', 'netto', 'deltaweight']].astype('int64')

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК ТАБЛИЦЫ 'scales_join'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы':  
 - количестве записей в таблице;
 - типы данных в таблице;
 - выведены для визуального контроля первые записи итоговых таблиц;

In [13]:
# Сводная информация по таблице 'scales_join'
scales_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39394 entries, 0 to 39393
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_scales    39394 non-null  int64         
 1   trnum_in     39394 non-null  int64         
 2   trnum_out    39394 non-null  int64         
 3   invnum_in    39394 non-null  int64         
 4   invnum_out   39394 non-null  int64         
 5   num_in       39394 non-null  int64         
 6   num_out      39394 non-null  int64         
 7   date_in      39394 non-null  datetime64[ns]
 8   date_out     37697 non-null  datetime64[ns]
 9   deltatetime  39394 non-null  object        
 10  tare_in      39394 non-null  int64         
 11  tare_out     39394 non-null  int64         
 12  brutto       39394 non-null  int64         
 13  netto        39394 non-null  int64         
 14  deltaweight  39394 non-null  int64         
dtypes: datetime64[ns](2), int64(12), object(1)
memory usa

In [79]:
# Общее количество записей таблицы 'scales_join'
len(scales_join)

39394

In [14]:
# Значения таблицы 'scales_join'
#filter_sj_1 = scales_join['id_scales'] == 5108
filter_sj_2 = scales_join['tare_in'] != scales_join['tare_out'] # фильтр - не совпадают масса пустого вагона на въезде и выезде
len((scales_join[filter_sj_2])['id_scales']) # количество записей соответствующих фильтру 'filter_sj_2'
#scales_join[filter_sj_1]
#scales_join[filter_sj_2]

#scales_join.iloc[2550:2600]
#scales_join.head(10)

2423

### БЛОК СОХРАНЕНИЯ ПРОМЕЖУТОЧНОГО ФАЙЛА CSV С ДАННЫМИ ТАБЛИЦЫ 'scales_join'  
---
 - на выходе получаем CSV файл

In [15]:
# Подготовить переменную с именем экспортируемого CSV файла
file_name_scales_join = '03__result-scales-correct-branch1.csv'
path_file_scales_join = Path(dir_path,'res', 'data', file_name_scales_join)

# Записать в данные в таблицу
scales_join.to_csv(str(path_file_scales_join), index=False, sep=';', encoding='utf-8')

### БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_join' И 'marriag' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ ЯВЛЯЮЩЕЙСЯ РЕШЕНИЕМ ЗАДАЧИ  
---
В данном блоке производится объединение записей таблиц scales_join и 'marriag' для получения итоговоЙ ТАБЛИЦЫ, которАЯ является решением задачи. Позже на ее основе будет получен итоговый выходной файл
правильных' записей о транзакции состава и всех входящих в него вагонов которые въехали на базу для налива и выехали после. Соединение производится по правилу LEFT JOIN с выполнением двойного условия. Так как 'pandas' не умеет выполнять оьбъединенеия по нечеткому соответствию (а именно таким получается второе условие), то для соединения была задействована дополнительная библиотека 'pandassql' которая работает поверх 'pandas' и перегоняет (динамически) данные в SQL базу данных SQLite, далее позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames.
Условия соединенеия подразумевают совпадения следующих полей обеих таблиц:  
 - 'invnum_out' и 'invnum';
 - нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
 - на выходе получаем таблицу 'join_hash_table';

In [16]:
# Подготовить SQL запрос для левого соединения таблиц 'scales_join' и 'marriag'

join_query_result_table = '''
    SELECT
        ROW_NUMBER() OVER(ORDER BY l.id_scales) + 1 as id_result
        ,l.id_scales   as id_scales
        ,r.id_marriag  as id_marriag
        ,l.trnum_in    as trnum_in
        ,l.trnum_out   as trnum_out
        ,l.invnum_in   as invnum_in
        ,l.invnum_out  as invnum_out
        ,l.num_in      as num_in
        ,l.num_out     as num_out
        ,l.date_in     as date_in
        ,l.date_out    as date_out
        ,r.bdatetime   as date_marriag
        ,l.deltatetime as deltatetime
        ,l.tare_in     as tare_in
        ,l.tare_out    as tare_out
        ,l.brutto      as brutto
        ,l.netto       as netto
        ,l.deltaweight as deltaweight
        ,r.nact        as nact
        ,r.reason      as reason
    FROM scales_join as l LEFT JOIN marriag as r
         ON (l.invnum_out = r.invnum)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.date_out, '-30 hours') as interger) AND cast(strftime('%s',l.date_out) as interger) )        
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
join_table = ps.sqldf(join_query_result_table, locals())

# Привести даты к типу datetime
join_table['date_in'] = pd.to_datetime(join_table['date_in'])
join_table['date_out'] = pd.to_datetime(join_table['date_out'])
join_table['date_marriag'] = pd.to_datetime(join_table['date_marriag'])

# Заменить NaN в столбце 'id_marriag' на 0
join_table['id_marriag'] = join_table['id_marriag'].fillna(0)

# Заменить тип столбца 'id_marriag' на int64
join_table['id_marriag'] = join_table['id_marriag'].astype('int64')

# Заменить None в столбце 'nact' на 0
join_table['nact'] = join_table['nact'].fillna(0)

# Заменить тип столбца 'nact' на int64
join_table['nact'] = join_table['nact'].astype('int64')


### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК РЕЗУЛЬТИРУЮЩЕЙ ТАБЛИЦЫ 'join_table'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы:
 - количестве записей в таблице;
 - типы данных в таблице;
 - выведены для визуального контроля первые записи таблицы;

In [14]:
# Сводная информация по таблице 'join_table'
join_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39500 entries, 0 to 39499
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id_result     39500 non-null  int64         
 1   id_scales     39500 non-null  int64         
 2   id_marriag    39500 non-null  int64         
 3   trnum_in      39500 non-null  int64         
 4   trnum_out     39500 non-null  int64         
 5   invnum_in     39500 non-null  int64         
 6   invnum_out    39500 non-null  int64         
 7   num_in        39500 non-null  int64         
 8   num_out       39500 non-null  int64         
 9   date_in       39500 non-null  datetime64[ns]
 10  date_out      37803 non-null  datetime64[ns]
 11  date_marriag  2590 non-null   datetime64[ns]
 12  deltatetime   39500 non-null  object        
 13  tare_in       39500 non-null  int64         
 14  tare_out      39500 non-null  int64         
 15  brutto        39500 non-null  int64 

In [17]:
# Общее количество записей таблицы 'join_table'
len(join_table)

39500

In [90]:
# Первые значения таблицы 'join_table'
join_table.head(5)

,id_result,id_scales,id_marriag,trnum_in,trnum_out,invnum_in,invnum_out,num_in,num_out,date_in,date_out,date_marriag,deltatetime,tare_in,tare_out,brutto,netto,deltaweight,nact,reason
0,2,0,0,688588,0,55344592,0,1,0,2022-01-26 04:19:00,NaT,NaT,,25300,0,0,0,0,0,None
1,3,0,0,688588,0,75081042,0,2,0,2022-01-26 04:19:00,NaT,NaT,,27350,0,0,0,0,0,None
2,4,0,0,688588,0,73071847,0,3,0,2022-01-26 04:19:00,NaT,NaT,,90250,0,0,0,0,0,None
3,5,0,0,688588,0,55303630,0,4,0,2022-01-26 04:19:00,NaT,NaT,,89950,0,0,0,0,0,None
4,6,0,0,688588,0,54253448,0,5,0,2022-01-26 04:20:00,NaT,NaT,,90800,0,0,0,0,0,None


### БЛОК СОХРАНЕНИЯ РЕШЕНИЯ ЗАДАЧИ ПО МЕТОДУ 'ВЕТКА 1' В CSV ФАЙЛ  
---
 - на выходе получаем CSV файл сгенерированный в соответствии с логикой 'ВЕТКА 1'

In [18]:
# Подготовить переменную с именем результирующего CSV файла
file_name_join = '05__result-table-branch1.csv'
path_file_join = Path(dir_path,'res', 'data', file_name_join)

# Записать в данные в файл
join_table.to_csv(str(path_file_join), index=False, sep=';', encoding='utf-8')


### БЛОК ТЕСТОВЫХ ФИЛЬТРОВ ДЛЯ ПРОСМОТРА РЕЗУЛЬТАТА В СООТВЕТСТВИИ С РАЗНЫМИ КРИТЕРИЯМИ  
---

In [19]:
# Набор условий для отбора показательных срезов для таблицы join_table
s_con_1 = abs(scales_out['deltaweight']) < scales_out['tare']*0.3 #дельта груза в пределах X% от массы вагона с пустой цистерной
s_con_2 = scales_out['deltaweight'] != scales_out['netto'] #моя дельта не равна дельте программы весов
s_con_3 = scales_out['netto'] != 0 #нетто не равно нулю
s_con_4 = scales_out['deltaweight']  < 0 #дельта груза отрицательная
s_con_5 = scales_out['deltaweight']  > 0 #дельта груза положительная
s_con_6 = scales_out['deltaweight'] != 0 #дельта груза не равна 0

In [20]:
# Набор условий для отбора показательных срезов для таблицы join_table
j_con_1 = join_table['date_marriag'].notnull() #отбор не нулевых данных
len(join_table[j_con_1])

2590

In [21]:
# Вывести значения нетто и дельты массы которые укладываются в s_con_1 % от массы порожнего вагона
scales_out[s_con_1 & s_con_6].head(5)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight
81,83,683563,75107136,1,2022-01-01 17:48:00,24800,25000,200,200
82,84,683563,50643741,2,2022-01-01 17:48:00,23900,23950,50,50
97,99,683563,51783413,17,2022-01-01 17:52:00,26900,27200,300,300
161,163,683643,54245592,1,2022-01-02 02:01:00,26900,27150,250,250
162,164,683643,54246533,2,2022-01-02 02:01:00,27350,27100,0,-250


In [22]:
#Вывести соотношение положительных и отрицательных дельт масс
prop = {'Количество +':len( scales_out[ s_con_1 & s_con_5 ] ), 
        'Количество -': len( scales_out[ s_con_4 ] ),
        '% +': round((len(scales_out[s_con_1 & s_con_5])/(len(scales_out[s_con_1 & s_con_5])+len(scales_out[ s_con_4 ])))*100),
        '% -': round((len(scales_out[ s_con_4 ])/(len(scales_out[s_con_1 & s_con_5])+len(scales_out[ s_con_4 ])))*100)}
prop

{'Количество +': 1524, 'Количество -': 1060, '% +': 59, '% -': 41}

In [151]:
# Скорость налива цистерн/12 часов при насосе 350 кг/мин
x=(12*0.35)
x

4.199999999999999

## ВЕТКА 2  
---
РЕШЕНИЕ ЗАДАЧИ ЧЕРЕЗ НЕ ЯВНО СУЩЕСТВУЮЩИЕ ПРИЗНАКИ. ДЛЯ РЕШЕНИЯ ВВОДИТСЯ И ИСПОЛЬЗУЕТСЯ ПОНЯТИЕ HASH ЗНАЧЕНИЯ

### ГРУППИРОВКА ДАННЫХ В DATAFRAMES scales_in И scales_out  
---
В данном блоке производится группировка записей таблиц 'scales_in' и 'scales_out' для получения сгруппированных по значению номера состава 'trnum' записей. Это нужно для достижения нескольких целей:
1. определения количества составов отдельно в таблицах **въехавших** и **выехавших** составов;
2. подготовка данных в обеих таблицах к такой форме хранения которая позволит эффективно вычислить **hash** значения;
3. вычисления комплексных характеристик каждого состава в обеих таблицах - **hash** значения;

Выработка требований к **hash** значению - нам потребуется определить такую группу характеристик которая атомарно представляет **уникальную** характеристику состава, причем должна имется 100% возможность получить такую группу как в таблице **въехавших** составах так и в таблице **выехавших** составов. Это требование должно не укоснительно выполняться, так как в дальнейшем полученный слепок состава - **hash** значение должно стать идентификатором по которому будет искаться совпадение и далее проводиться объединение данных по идентичным составам из обеих таблиц.  
В качестве такого группового признака который имперически определен как абсолютно не повторяемый ни при каких условиях - последовательность значений номеров состава а также (одновременно) позиционный номер вагона в составе, т.е. математическая последовательность значений номеров вагонов. Для упрощения кода, фактор порядкового номера был заменен на не явный, а именно перед вычислением **hash** значения номера вагонов превращены в текстовые значения и произведена конкатенация строго в той последовательности в которой вагоны шли в составе. В итоге входным параметром функции-генератора **hash** значения стала тестовая строка состаящая из непрерывно сцепленных значений текстовых эквивалентов номеров вагонов.  
В качестве функции-генератора **hash** значения применена - **MD5**

In [48]:
# Вывести первые 3 записи (до группировки) комплементарных групп которе далее должны сгруппироваться а далее объединиться
# это требуется для визуального контроля
scales_in[ scales_in['trnum'] == 683482 ].sort_values(by='num').head(3)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight


In [49]:
# Вывести первые 3 записи (до группировки) комплементарных групп которе далее должны сгруппироваться а далее объединиться
# это требуется для визуального контроля
scales_out[ scales_out['trnum'] == 683563 ].sort_values(by='num').head(3)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight


In [50]:
# Произвести группировку по номеру состава ('trnum')
gr_scales_in = scales_in.groupby('trnum')
gr_scales_out = scales_out.groupby('trnum')

In [51]:
print(gr_scales_in.ngroups, '- количество уникальных составов типа - IN') # количество уникальных составов типа - IN 
print(gr_scales_out.ngroups,'- количество уникальных составов типа - OUT') # количество уникальных составов типа - OUT 
print(gr_scales_in.ngroups - gr_scales_out.ngroups, '- разница IN и OUT составов') # разница  IN и OUT составов

178 - количество уникальных составов типа - IN
154 - количество уникальных составов типа - OUT
24 - разница IN и OUT составов


### БЛОК ГЕНЕРАЦИИ HASH ЗНАЧЕНИЙ  
---

In [52]:
# Функция вычисляет hash значение группы
def HashRow(_df):
    
    hash_total_str = ''
    
    df = _df.sort_values(by='num', ascending=True)
    # Перебрать все строки текущей группы, получить текстовый объект hash всех номеров вагонов группы
    for data in df.itertuples():
        hash_total_str += str(data[3])
    
    hash_object = hs.md5(hash_total_str.encode('utf-8'))
    #return int(hash_object.hexdigest(), 16)
    return hash_object.hexdigest()

# Функция выпоняет перебор всех групп и вызывает функцию вычисления hash значения группы
def HashGroup(_df):
    
    hash_dict = {} # Словарь хранящий пару 'Номер состава: hash состава'
    cycle = 0
    
    # Перебраить все группы сгруппированого DataFrame
    for name, group in _df:
        if cycle>0:
            break
        hash_dict[name] = HashRow(group) # Для каждой группы вызвать hash функцию
    return hash_dict

In [53]:
# Сгенерировать словари с hash значениями групп составовs
hash_dict_in  = HashGroup(gr_scales_in)
hash_dict_out = HashGroup(gr_scales_out)

# Преобразовать dictonari в DataFrame
hash_df_in = pd.DataFrame(hash_dict_in.items(), columns=['trnum', 'hash_group'])
hash_df_out = pd.DataFrame(hash_dict_out.items(), columns=['trnum', 'hash_group'])

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК РЕЗУЛЬТИРУЮЩЕЙ ТАБЛИЦ HASH ЗНАЧЕНИЙ 'hash_df_in' И 'hash_df_out'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы:
 - количестве записей в таблице;
 - выведены для визуального контроля первые записи таблицы;

In [54]:
# Вывести первые записи таблицы 'hash_df_in' для визуального контроля
hash_df_in.head(3)

,trnum,hash_group
0,761304,754875f74fbd809411651574e4ddeb73
1,761385,d35ed09492358eebca63cd5b274108b0
2,761426,e37b33865e92228d247b74e867ffd5b9


In [19]:
# Вывести первые записи таблицы 'hash_df_out' для визуального контроля
hash_df_out.head(3)

,trnum,hash_group
0,761345,2eaaa69c379869720e5fad57e6afa228
1,761426,754875f74fbd809411651574e4ddeb73
2,761586,2fb734e9b08d3ea1a276810389556334


### БЛОК СОЕДИНЕНИЯ ТАБЛИЦ 'scales_in','scales_out' С СООТВЕТСТВУЮЩИМИ ТАБЛИЦАМИ HASH ЗНАЧЕНИЙ  
---

In [55]:
# Выполнить левое соединение таблиц 'scales_hash_in', 'scales_hash_out' с соответствующими таблицами hash значений
scales_hash_in = scales_in.merge(hash_df_in, on='trnum', how='left')
scales_hash_out = scales_out.merge(hash_df_out, on='trnum', how='left')

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК РЕЗУЛЬТИРУЮЩЕЙ ТАБЛИЦ HASH ЗНАЧЕНИЙ 'scales_hash_in' И 'scales_hash_out'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы:
 - количестве записей в таблице;
 - выведены для визуального контроля первые записи таблицы;

In [56]:
# Вывести первые записи для визуального контроля
scales_hash_in.head(3)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight,hash_group
0,2,761304,73915704,41,2023-01-01 01:41:20,0,27100,0,27100,754875f74fbd809411651574e4ddeb73
1,3,761304,50919992,40,2023-01-01 01:41:36,0,26350,0,26350,754875f74fbd809411651574e4ddeb73
2,4,761304,50787738,39,2023-01-01 01:41:57,0,27550,0,27550,754875f74fbd809411651574e4ddeb73


In [38]:
# Вывести количество записей таблицы 'scales_hash_in'
len(scales_hash_in)

6527

In [39]:
# Вывести первые записи для визуального контроля
scales_hash_out.head(3)

,id_scales,trnum,invnum,num,bdatetime,tare,brutto,netto,deltaweight,hash_group
0,43,761345,57187395,1,2023-01-01 03:35:54,27850,28050,200,200,2eaaa69c379869720e5fad57e6afa228
1,44,761345,54240205,2,2023-01-01 03:36:11,27100,26950,0,-150,2eaaa69c379869720e5fad57e6afa228
2,45,761345,73090953,3,2023-01-01 03:36:27,27600,88900,61300,61300,2eaaa69c379869720e5fad57e6afa228


In [40]:
# Вывести количество записей таблицы 'scales_hash_out'
len(scales_hash_out)

6213

### БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_hash_in' И 'scales_hash_out' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ 'scales_join_hash' КОТОРАЯ ХРАНИТ ОБЪЕДИНЕНЫЕ ЗАПИСИ ВЪЕХАВШИХ И ВЫЕХАВШИХ СОСТАВОВ.  
#### АНАЛОГИЧНАЯ ТАБЛИЦА ПОЛУЧЕНА РАНЕЕ В ВЕТКЕ 1 ПО ОТЛИЧНОМУ ОТ ВЕТКЕ 2 АЛГОРИТМУ, А ИМЕННО БЕЗ ИСПОЛЬЗОВАНИЯ HASH ЗНАЧЕНИЙ
---
В данном блоке производится соединение записей таблиц 'scales_hash_in' и 'scales_hash_out'. Позже на основе результирующей таблицы 'scales_join_hash' будет записан результирующий выходной файл CSV файл, являющийся решением задачи.  
Соединение производится по правилу LEFT JOIN с выполнением тройного условия. В данном случае у нас все три условия **четкие**, поэтому можно произвести объединение средствами библиотеки 'pandas', но из за экономии времени, единообразности технических решений ранее уже примененных в данной программе, а также гибкости компоновки сстолбцов итоговой таблицы, соединение будет производится средствами библиотеки 'pandassql', которая работает поверх 'pandas' и перегоняет (динамически) данные в SQL базу данных SQLite, далее позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames.
Условия соединенеия подразумевают совпадения следующих полей обеих таблиц:  
 - 'invnum_out' и 'invnum';
 - 'num' обеих таблиц;
 - 'hash_group' обеих таблиц;
 - на выходе получаем таблицу 'scales_join_hash';

In [57]:
# Преобразуем тип 'hash_group' в string, потому что pandassql не умеет конвертировать int large в целочисленные типы SQLite

# Данное действие требуется только для варианта программы где 'hash' значение конвертировалось в int larg, если работа ведется 
# с текстовыми 'hash' значениями то оно не нужно, поэтому эти строки кода могут быть закоментированы

#scales_hash_in['hash_group'] = scales_hash_in['hash_group'].astype(str)
#scales_hash_out['hash_group'] = scales_hash_out['hash_group'].astype(str)

# Подготовить SQL запрос для левого соединения таблиц 'scales_hash_in' и 'scales_hash_out'
join_query_scales_hash = '''
    SELECT
        l.id_scales     as id_scales
        ,l.trnum         as trnum_in
        ,r.trnum         as trnum_out
        ,l.invnum        as invnum_in
        ,r.invnum        as invnum_out
        ,l.num           as num_in
        ,l.num           as num_out
        ,l.bdatetime     as date_in
        ,r.bdatetime     as date_out
        ,l.brutto        as tare_in
        ,r.tare          as tare_out
        ,r.brutto        as brutto
        ,r.netto         as netto
        ,r.deltaweight   as deltaweight
        ,l.hash_group    as hash_group
    FROM scales_hash_in as l LEFT JOIN scales_hash_out as r
         ON     (l.invnum     = r.invnum)
            AND (l.num        = r.num)
            AND (l.hash_group = r.hash_group)
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
scales_join_hash = ps.sqldf(join_query_scales_hash, locals())

# Привести даты к типу datetime
scales_join_hash['date_in']  = pd.to_datetime(scales_join_hash['date_in'])
scales_join_hash['date_out'] = pd.to_datetime(scales_join_hash['date_out'])

# Вставить столбец 'deltatetime' с данными разницы времени (дени - часа - минуты - секунды) между событием 'въехал' и событием 'выехал'
scales_join_hash.insert(9, 'deltatetime', scales_join_hash['date_out']-scales_join_hash['date_in'])

# Нормировать разницу к формату 00,00 (часы и доли часа)
scales_join_hash['deltatetime'] = (scales_join_hash['deltatetime']/pd.Timedelta('1 hour')).round(2)

# Преобразовать 'deltatetime' к string и заменить '.' на ',' т.к. импортирует с '.' для дальнейшего использования совместно с Excel
scales_join_hash['deltatetime'] = scales_join_hash['deltatetime'].astype(str).str.replace('.', ',', regex=False)

# Заменить NaN в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на 0
scales_join_hash[['trnum_out'
                 ,'invnum_out'
                 ,'num_out'
                 ,'deltatetime'
                 ,'tare_out'
                 ,'brutto'
                 ,'netto'
                 ,'deltaweight']] = scales_join_hash[['trnum_out', 'invnum_out', 'num_out' ,'deltatetime', 'tare_out', 'brutto', 'netto', 'deltaweight']].fillna(0)

# Заменить тип в столбцах 'trnum_out', 'invnum_out', 'num_out', 'brutto', 'netto', 'deltaweigth' на int64
scales_join_hash[['trnum_out'
                 ,'invnum_out'
                 ,'num_out'
                 ,'tare_out'
                 ,'brutto'
                 ,'netto'
                 ,'deltaweight']] = scales_join_hash[['trnum_out', 'invnum_out', 'num_out', 'tare_out','brutto', 'netto', 'deltaweight']].astype('int64')

### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК РЕЗУЛЬТИРУЮЩЕЙ ТАБЛИЦЫ 'scales_join_hash'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы:
 - типы данных таблицы;
 - количестве записей в таблице;
 - выведены для визуального контроля первые записи таблицы;

In [58]:
# Вывести информацию о типах
scales_join_hash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6568 entries, 0 to 6567
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_scales    6568 non-null   int64         
 1   trnum_in     6568 non-null   int64         
 2   trnum_out    6568 non-null   int64         
 3   invnum_in    6568 non-null   int64         
 4   invnum_out   6568 non-null   int64         
 5   num_in       6568 non-null   int64         
 6   num_out      6568 non-null   int64         
 7   date_in      6568 non-null   datetime64[ns]
 8   date_out     6051 non-null   datetime64[ns]
 9   deltatetime  6568 non-null   object        
 10  tare_in      6568 non-null   int64         
 11  tare_out     6568 non-null   int64         
 12  brutto       6568 non-null   int64         
 13  netto        6568 non-null   int64         
 14  deltaweight  6568 non-null   int64         
 15  hash_group   6568 non-null   object        
dtypes: dat

In [59]:
# Вывести информацию о количестве записей
len(scales_join_hash)

6568

In [60]:
# Вывести первые значения таблицы
scales_join_hash.head(3)

,id_scales,trnum_in,trnum_out,invnum_in,invnum_out,num_in,num_out,date_in,date_out,deltatetime,tare_in,tare_out,brutto,netto,deltaweight,hash_group
0,2,761304,0,73915704,0,41,41,2023-01-01 01:41:20,NaT,nan,27100,0,0,0,0,754875f74fbd809411651574e4ddeb73
1,3,761304,0,50919992,0,40,40,2023-01-01 01:41:36,NaT,nan,26350,0,0,0,0,754875f74fbd809411651574e4ddeb73
2,4,761304,0,50787738,0,39,39,2023-01-01 01:41:57,NaT,nan,27550,0,0,0,0,754875f74fbd809411651574e4ddeb73


### БЛОК СОХРАНЕНИЯ ПРОМЕЖУТОЧНОГО ФАЙЛА CSV С ДАННЫМИ ТАБЛИЦЫ 'scales_join'  
---
 - на выходе получаем CSV файл сгенерированный в соответствии с логикой 'ВЕТКА 2'

In [61]:
# Подготовить переменную с именем экспортируемого CSV файла 
file_name_scales_join_hash = '04__result-scales-correct-branch2.csv'
path_file_scales_join_hash = Path(dir_path,'res', 'data', file_name_scales_join_hash)

# Записать в данные в таблицу
scales_join_hash.to_csv(str(path_file_scales_join_hash), index=False, sep=';', encoding='utf-8')

### БЛОК СОЕДИНЕНИЯ ЗАПИСЕЙ 'scales_join_hash' И 'marriag' ДЛЯ ПОЛУЧЕНИЯ ТАБЛИЦЫ ЯВЛЯЮЩЕЙСЯ РЕШЕНИЕМ ЗАДАЧИ  
---
В данном блоке производится объединение записей таблиц scales_join_hash и 'marriag' для получения итоговоЙ ТАБЛИЦЫ, которая является решением задачи. Позже на ее основе будет получен итоговый выходной файл
правильных' записей о транзакции состава и всех входящих в него вагонов которые въехали на базу для налива и выехали после. Соединение производится по правилу LEFT JOIN с выполнением двойного условия. Так как 'pandas' не умеет выполнять оьбъединенеия по нечеткому соответствию (а именно таким получается второе условие), то для соединения была задействована дополнительная библиотека 'pandassql' которая работает поверх 'pandas' и перегоняет (динамически) данные в SQL базу данных SQLite, далее позволяет выполнить преобразование с помощью команд SQL, а после обратно сохранить в объекты 'pandas' - DataFrames.
Условия соединенеия подразумевают совпадения следующих полей обеих таблиц:  
 - 'invnum_out' и 'invnum';
 - нечеткое совпадение временного поля 'bdatetime' (по принципу оконной функции);
 - на выходе получаем таблицу 'join_table';

In [62]:
# Подготовить SQL запрос для левого соединения таблиц 'scales_join_hash' и 'marriag'

join_hash_query_result_table = '''
    SELECT
        ROW_NUMBER() OVER(ORDER BY l.id_scales) + 1 as id_result
        ,l.id_scales   as id_scales
        ,r.id_marriag  as id_marriag
        ,l.trnum_in    as trnum_in
        ,l.trnum_out   as trnum_out
        ,l.invnum_in   as invnum_in
        ,l.invnum_out  as invnum_out
        ,l.num_in      as num_in
        ,l.num_out     as num_out
        ,l.date_in     as date_in
        ,l.date_out    as date_out
        ,r.bdatetime   as date_marriag
        ,l.deltatetime as deltatetime
        ,l.tare_in     as tare_in
        ,l.tare_out    as tare_out
        ,l.brutto      as brutto
        ,l.netto       as netto
        ,l.deltaweight as deltaweight
        ,r.nact        as nact
        ,r.reason      as reason
        ,l.hash_group  as hash_group
    FROM scales_join_hash as l LEFT JOIN marriag as r
         ON (l.invnum_out = r.invnum)
            AND (cast(strftime('%s',r.bdatetime) as interger)
                BETWEEN cast(strftime('%s',l.date_out, '-30 hours') as interger) AND cast(strftime('%s',l.date_out) as interger) )        
'''

# Выполнить ранее подготовленный запрос средствами библиотеки 'pandasql', получить результирующую таблицу
join_hash_table = ps.sqldf(join_hash_query_result_table, locals())

# Привести даты к типу datetime
join_hash_table['date_in'] = pd.to_datetime(join_hash_table['date_in'])
join_hash_table['date_out'] = pd.to_datetime(join_hash_table['date_out'])
join_hash_table['date_marriag'] = pd.to_datetime(join_hash_table['date_marriag'])

# Заменить NaN в столбце 'id_marriag' на 0
join_hash_table['id_marriag'] = join_hash_table['id_marriag'].fillna(0)

# Заменить тип столбца 'id_marriag' на int64
join_hash_table['id_marriag'] = join_hash_table['id_marriag'].astype('int64')

# Заменить None в столбце 'nact' на 0
join_hash_table['nact'] = join_hash_table['nact'].fillna(0)

# Заменить тип столбца 'nact' на int64
join_hash_table['nact'] = join_hash_table['nact'].astype('int64')


### БЛОК ВИЗУАЛИЗАЦИИ ХАРАКТЕРИСТИК РЕЗУЛЬТИРУЮЩЕЙ ТАБЛИЦЫ 'join_hash_table'  
---
В данном блоке выводятся на печать основные интегральные характеристики таблицы:
 - количестве записей в таблице;
 - типы данных в таблице;
 - выведены для визуального контроля первые записи таблицы;


In [63]:
# Сводная информация по таблице 'join_table'
join_hash_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6596 entries, 0 to 6595
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id_result     6596 non-null   int64         
 1   id_scales     6596 non-null   int64         
 2   id_marriag    6596 non-null   int64         
 3   trnum_in      6596 non-null   int64         
 4   trnum_out     6596 non-null   int64         
 5   invnum_in     6596 non-null   int64         
 6   invnum_out    6596 non-null   int64         
 7   num_in        6596 non-null   int64         
 8   num_out       6596 non-null   int64         
 9   date_in       6596 non-null   datetime64[ns]
 10  date_out      6079 non-null   datetime64[ns]
 11  date_marriag  375 non-null    datetime64[ns]
 12  deltatetime   6596 non-null   object        
 13  tare_in       6596 non-null   int64         
 14  tare_out      6596 non-null   int64         
 15  brutto        6596 non-null   int64   

In [64]:
# Общее количество записей таблицы 'join_table'
len(join_hash_table)

6596

In [46]:
# Первые значения таблицы 'join_table'
join_hash_table.head(2)

,id_result,id_scales,id_marriag,trnum_in,trnum_out,invnum_in,invnum_out,num_in,num_out,date_in,...,date_marriag,deltatetime,tare_in,tare_out,brutto,netto,deltaweight,nact,reason,hash_group
0,2,2,0,683482,683563,73218588,73218588,40,40,2022-01-01 04:46:00,...,NaT,"13,2",26400,26400,86900,60500,60500,0,None,e014f131431942ef0ffe305e99d6749d
1,3,3,0,683482,683563,51920114,51920114,39,39,2022-01-01 04:46:00,...,NaT,"13,2",25950,25950,88000,62050,62050,0,None,e014f131431942ef0ffe305e99d6749d


### БЛОК СОХРАНЕНИЯ РЕШЕНИЯ ЗАДАЧИ ПО МЕТОДУ 'ВЕТКА 2' В CSV ФАЙЛ  
---  
 - на выходе получаем CSV файл сгенерированный в соответствии с логикой 'ВЕТКА 2'



In [65]:
# Подготовить переменную с именем результирующего CSV файла
file_name_join_hash_table = '06__result-table-branch2.csv'
path_file_join_hash_table = Path(dir_path,'res', 'data', file_name_join_hash_table)

# Записать в данные в файл
join_hash_table.to_csv(str(path_file_join_hash_table), index=False, sep=';', encoding='utf-8')